In [4]:
import os
import numpy as np
from scipy.optimize import nnls
from pbwrap.utils import open_image

# Load example image

In [5]:
PATH = "/home/floric/Documents/fish_images/"
images_filenames = os.listdir(PATH)
images_filenames

filename = images_filenames[0]

In [6]:
image = open_image(PATH + filename).squeeze()
image = image[0]
image.shape

(19, 626, 626)

# 1. INPUT

From function call we need :  
* Images to correct : list[np.ndarray]
* exposure_time : int or list[int]

from this to initialize our problem we need to flatten images to correct on put those into a MxN matrix where *M* is the number of image in the list and *N* the number of pixel in this images (must be the same).
Then create the diagonal matrix MxM 

In [29]:
def _initialize_matrix(
    images : np.ndarray | list[np.ndarray],
    exposure_time : int | list[int]
) :

    if isinstance(images, list) :
        if not all([isinstance(im,np.ndarray) for im in images]) : raise TypeError("All list element must be ndarrays.")
        length_list = [im.size for im in images]
        if min(length_list) != max(length_list) : raise ValueError("Passed images must have same shape")
    elif isinstance(images, np.ndarray) :
        images = [images]
    else :
        raise TypeError("Wrong type for images. Expected list or ndarray got {}".format(type(images)))
    
    if isinstance(exposure_time, list) :
        if not all([isinstance(im,int) for im in images]) : raise TypeError("All list element must be ints.")
    elif isinstance(exposure_time, int) :
        exposure_time = [exposure_time]
    else :
        raise TypeError("Wrong type for exposure time. Expected int or list got {}".format(type(exposure_time)))
        
    # Def A matrix : observed signal matrix
    flatten_images = [im.flatten() for im in images]
    observed_matrix = np.array(flatten_images, dtype=np.uint)

    images_number, pixel_number =  observed_matrix.shape

    # Def E matrix : Exposure time matrix
    exposure_matrix = np.diag(exposure_time)

    #Init matrix C : system output
    decomposed_signal_matrix = np.ones(shape=(2,pixel_number), dtype=np.uint)

    #Init matrix B : linear coefficient
    linear_coef_matrix = np.ones(shape=(images_number,2),dtype=np.uint)

    #Init matrix D : Dark current
    dark_current_matrix = np.ones(shape=(images_number,pixel_number))

    #Init matrix N : Noise
    noise_matrix = np.ones(shape=(images_number,pixel_number))


    return observed_matrix, exposure_matrix, decomposed_signal_matrix, linear_coef_matrix, dark_current_matrix, noise_matrix

In [31]:
A,E,B,C,D,N = _initialize_matrix(images=image, exposure_time= 100)
np.dot(E,np.dot(C,B) + D) + N

array([[301., 301., 301., ..., 301., 301., 301.]], shape=(1, 7445644))